In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #Input = 28X28X1 - Output = 26X26X16 - RF = 3
        self.conv1 = nn.Conv2d(1, 16, 3, bias=False) 
        self.bn1 = nn.BatchNorm2d(16, affine=True)
        self.dropout1 = nn.Dropout2d(0.1)
         #Input = 26X26X16 - Output = 24X24X16 - RF = 5
        self.conv2 = nn.Conv2d(16, 16, 3, bias=False)
        self.bn2 = nn.BatchNorm2d(16, affine=True)
        self.dropout2 = nn.Dropout2d(0.1)

        #Input = 24X24X16 - Output = 22X22X16 - RF = 7
        self.conv3 = nn.Conv2d(16, 16, 3, bias=True) 
        self.bn3 = nn.BatchNorm2d(16, affine=True)
        self.dropout3 = nn.Dropout2d(0.1)
        #Input = 22X22X16 - Output = 11X11X16 - RF = 8
        self.pool1 = nn.MaxPool2d(2, 2) 
        
        #Input = 11X11X16 - Output = 9X9X16- RF = 14
        self.conv4 = nn.Conv2d(16, 16, 3, bias=False) 
        self.bn4 = nn.BatchNorm2d(16, affine=True)
        self.dropout4 = nn.Dropout2d(0.1)

        #Input = 9X9X16 - Output = 7X7X16 - RF = 16
        self.conv5 = nn.Conv2d(16, 16, 3, bias=False) 
        self.bn5 = nn.BatchNorm2d(16, affine=True)
        self.dropout5 = nn.Dropout2d(0.1)

        #Input = 7X7X16 - Output = 7X7X16 - RF = 18
        self.conv6 = nn.Conv2d(16, 16, 3, bias=False, padding=1) 
        self.bn6 = nn.BatchNorm2d(16, affine=True)
        self.dropout6 = nn.Dropout2d(0.1)
        
        self.avg_pool = nn.AvgPool2d(7) #Output = 1X1X16
        self.pointwise3 = nn.Conv2d(16, 10, 1, bias=False)
        
    def forward(self, x):
        x = self.dropout1(self.bn1(F.relu(self.conv1(x))))
        x = self.dropout2(self.bn2(F.relu(self.conv2(x))))
        x = self.dropout3(self.bn3(F.relu(self.conv3(x))))
        x = self.pool1(x)
        x = self.dropout4(self.bn4(F.relu(self.conv4(x))))
        x = self.dropout5(self.bn5(F.relu(self.conv5(x))))
        x = self.dropout6(self.bn6(F.relu(self.conv6(x))))
        # x = self.conv7(x)
        #print(x.shape)
        x = self.avg_pool(x)
        #print(x.shape)
        #x = x.view(-1, 32)
        x = self.pointwise3(x)
        #print(x.shape)
        x = x.view(-1, 10)
        return F.log_softmax(x, dim=1)

In [3]:
#!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             144
       BatchNorm2d-2           [-1, 16, 26, 26]              32
         Dropout2d-3           [-1, 16, 26, 26]               0
            Conv2d-4           [-1, 16, 24, 24]           2,304
       BatchNorm2d-5           [-1, 16, 24, 24]              32
         Dropout2d-6           [-1, 16, 24, 24]               0
            Conv2d-7           [-1, 16, 22, 22]           2,320
       BatchNorm2d-8           [-1, 16, 22, 22]              32
         Dropout2d-9           [-1, 16, 22, 22]               0
        MaxPool2d-10           [-1, 16, 11, 11]               0
           Conv2d-11             [-1, 16, 9, 9]           2,304
      BatchNorm2d-12             [-1, 16, 9, 9]              32
        Dropout2d-13             [-1, 16, 9, 9]               0
           Conv2d-14             [-1, 1

In [4]:
torch.manual_seed(1)
batch_size = 32

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [5]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
             # sum up batch loss
            test_loss += F.nll_loss(output, target, reduction='sum').item() 
            # get the index of the max log-probability
            pred = output.argmax(dim=1, keepdim=True)  
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(0, 19):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/1875 [00:00<?, ?it/s]


Test set: Average loss: 0.0486, Accuracy: 9858/10000 (98.58%)



  0%|          | 0/1875 [00:00<?, ?it/s]


Test set: Average loss: 0.0361, Accuracy: 9877/10000 (98.77%)



  0%|          | 0/1875 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0186, Accuracy: 9939/10000 (99.39%)



  0%|          | 0/1875 [00:00<?, ?it/s]


Test set: Average loss: 0.0181, Accuracy: 9943/10000 (99.43%)



loss=0.05833295360207558 batch_id=1874: 100%|██████████| 1875/1875 [00:21<00:00, 85.43it/s]



Test set: Average loss: 0.0205, Accuracy: 9941/10000 (99.41%)

